In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import random
import sys
import pickle
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
import seaborn as sns

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skorch import NeuralNetRegressor

from sklearn.model_selection import GridSearchCV
from collections import defaultdict
from sklearn import metrics
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  

cpu


C:\Users\Erika\anaconda3\envs\Python3-7\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def draw_heatmap_2d(training_errors, gamma_list, C_list, title, xlabel, ylabel, large=True):
    if large:
        plt.figure(figsize = (10,6))
    ax = sns.heatmap(training_errors, annot=True, fmt='.3f', 
                     xticklabels=gamma_list, yticklabels=C_list)
    ax.collections[0].colorbar.set_label("error")
    ax.set(xlabel=xlabel, ylabel=ylabel)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.title(title)
    plt.show()

## Models

In [3]:
class Net(nn.Module):  
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(20*32*161, 100) # Get the vector of the layer right before the final fc layer and plot tfidf
        self.fc2 = nn.Linear(100, 10) 
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 20*32*161)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [4]:
def cnn(X_train, Y_train, X_test, Y_test, draw=False):
    net = NeuralNetRegressor(Net, max_epochs=100, lr=0.001, verbose=1)
    
    params = {
        #'lr': [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3],
        #'max_epochs': list(range(500,5500, 500))
        'lr': [0.001],
        'max_epochs': [1, 3]
    }

    grid_search = GridSearchCV(net, params, refit=False, scoring='r2', verbose=1, cv=10)
    
    
    #print(X_train.shape)
    print(Y_train.shape)
    
    grid_search.fit(X_train, Y_train)
    
    """
    cross_val_errors = 1 - grid_search.cv_results_['mean_test_score'].reshape(1,2)
    if draw:
        draw_heatmap_2d(cross_val_errors, k_list, metric_list, title='cross-validation error w.r.t k and distance metric', ylabel='metric', xlabel='k', large=True)
    """
    
    best_params = grid_search.best_params_
    print("Best params: {}".format(best_params))

    best_test_accuracy = (Y_test == grid_search.best_estimator_.predict(X_test)).sum() / len(Y_test)
    best_train_accuracy = (Y_train == grid_search.best_estimator_.predict(X_train)).sum() / len(Y_train)

    return best_train_accuracy, best_test_accuracy

## Load preprocessed data

In [5]:
class GTZANDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][0]
        y = self.data[idx][1]
        
        if self.transform:
            x = self.transform(x)
        
        x = torch.Tensor(x)

        return (x, y)

In [6]:
batch_size = 1


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

raw_trainset = pickle.load(open('data/GTZAN.train', 'rb'))
trainset = GTZANDataset(raw_trainset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

raw_testset = pickle.load(open('data/GTZAN.test', 'rb'))
testset = GTZANDataset(raw_testset)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


classes = ('blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock')

In [7]:
"""
net = Net()
net.to(device)


loss_func = nn.CrossEntropyLoss()  
# We use stochastic gradient descent (SGD) as optimizer.
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 
"""

'\nnet = Net()     # Create the network instance.\nnet.to(device)  # Move the network parameters to the specified device.\n'

## Hyperparameter Optimization

In [9]:
dataset_names = ['GTZAN']
datasets = [trainset]
models = [cnn]
model_names = ['CNN']
partition = 0.8
num_trials = 3

### Train

In [18]:
for i, dataset in enumerate(datasets):      
    print('DATASET: ' + dataset_names[i])
    print('% TRAINING: ' + str(partition))
    print('-------------------------------------')

    model_acc_sums = np.zeros((len(models), 2))
    for k in range(num_trials):
        print('TRIAL: %d' % (k+1))
        print('---------')
        
        XY_train = list(zip(*raw_trainset))
        X_train = np.array(XY_train[0])
        Y_train = np.array(XY_train[1]).reshape(-1, 1)
        
        XY_test = list(zip(*raw_testset))
        X_test = np.array(XY_train[0])
        Y_test = np.array(XY_train[1]).reshape(-1, 1)
        
        for j, model in enumerate(models):
            print('MODEL: %s' % model_names[j])

            train_acc, test_acc = model(X_train, Y_train, X_test, Y_test, draw=True)

            print('TRIAL TRAIN ACCURACY FOR DATASET FOR MODEL: ' + str(train_acc))
            print('TRIAL TEST ACCURACY FOR DATASET FOR MODEL: ' + str(test_acc))
            print()

            model_acc_sums[j][0] += train_acc
            model_acc_sums[j][1] += test_acc

    print('-----RESULTS-----')
    print('DATASET: ' + dataset_names[i])
    print('% TRAINING: ' + str(partition))
    print()
    for j, model in enumerate(model_names):
        print('MODEL: %s' % model_names[j])
        avg_train_acc = round(model_acc_sums[j][0] / num_trials, 2)
        avg_test_acc = round(model_acc_sums[j][1] / num_trials, 2)

        print('AVG TRAIN ACCURACY FOR %s WITH %s DATASET WITH %s TRAINING SPLIT: %s' % (model_names[j], dataset_names[i], partition, str(avg_train_acc)))
        print('AVG TEST ACCURACY FOR %s WITH %s DATASET WITH %s TRAINING SPLIT: %s' % (model_names[j], dataset_names[i], partition, str(avg_test_acc)))
        print()
    print()

DATASET: GTZAN
% TRAINING: 0.8
-------------------------------------
TRIAL: 1
---------
MODEL: CNN
(800, 1)
Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best params: {'lr': 0.001, 'max_epochs': 1}


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   11.5s finished


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'